In [1]:
# Importiere Bibliotheken
import os
import re
import urllib
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords

from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from keras.layers import Flatten
from keras.layers import MaxPooling1D
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Concatenate

In [2]:
# Kodierte Daten einlesen
data_labeled = pd.read_pickle('data_modified/tweets_labeled.pkl')
data_labeled.head()

,User,Tweet,Retweet,RT_from,Orig_Tweet,Tweet_Participants,Reply,Text,Tags,Lang,Link,Label
Datetime,,,,,,,,,,,,
2018-01-18 17:07:00,@GeroldSchlegel,RT @redder66: @NatalieRickli @NoBillag Die Unt...,True,@redder66,False,"[@NatalieRickli, @NoBillag, @SVPch]",False,Die Unterstützung der für die #NoBillag ergibt...,"[NoBillag, NeinzuNoBillag]",de,[],1
2017-12-23 13:53:00,@derlamentierer,@ProBillag auch sie haben das konzept von soli...,False,NaN,False,[@ProBillag],True,auch sie haben das konzept von solidarität nic...,"[nobillag, jazunobillag, NeinzuNoBillag]",de,[https://t.co/iFLeHBuSFS],6
2017-12-05 19:47:00,@FlorianSchwab,RT @VitalSteiner: Was die Gegner von #NoBillag...,True,@VitalSteiner,False,[],False,Was die Gegner von #NoBillag mit solchen Bilde...,[NoBillag],de,[https://t.co/IKOSeftUlm],6
2018-01-17 07:08:00,@Schweizer2018,RT @realDanielVozar: Wenn #NoBillag abgelehnt ...,True,@realDanielVozar,False,[@LHeimgartner],False,"Wenn #NoBillag abgelehnt wird, wird sich gar n...","[NoBillag, JazuNoBillag]",de,[],5
2018-02-19 09:52:00,@PilgramsWords,RT @_macmike: #StarkesStück https://t.co/2L7uG...,True,@_macmike,False,"[@Blickch, @20min, @bazonline]",False,#StarkesStück #AfD #SVP #ARD #ZDF #ORF #SRF #...,"[StarkesStück, AfD, SVP, ARD, ZDF, ORF, SRF, s...",de,"[https://t.co/2L7uGmmVH0, https://t.co/5Wf6qJ1...",0


In [3]:
# Funktionen definieren um Stoppworte zu entfernen & Daten weiter zu bereinigen
nltk.download('stopwords')
stopword_set = set(stopwords.words("german"))
def preprocess(raw_text):
    res = raw_text
    res = res.replace('ä', 'ae')
    res = res.replace('ö', 'oe')
    res = res.replace('ü', 'ue')
    res = res.replace('Ä', 'Ae')
    res = res.replace('Ö', 'Oe')
    res = res.replace('Ü', 'Ue')
    res = res.replace('ß', 'ss')
    stopword_set = set(stopwords.words("german"))
    return " ".join([i for i in re.sub(r'[^a-zA-Z\s]', "", res).lower().split() if i not in stopword_set])

[nltk_data] Downloading package stopwords to C:\Users\Sven
[nltk_data]     Cornehls\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Daten bereinigen und X und y Variablen legen
texts=[]
for txt in data_labeled['Text']:
    txt=preprocess(txt)
    texts.append(txt)
labels = data_labeled['Label'].values

In [5]:
# CNN und Word2Vec-Parameter definieren
MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 300

EMBEDDING_FILE = "Path to: ...\german.model" #Muss wieder angepasst werden
category_index = {"Kann nicht zugordnet werden": 0,
                  "Pressefreiheit":1,
                  "Kommerzialisierung":2,
                  "Mediale Grundversorgung":3,
                  "Finanzen Haushalte":4,
                  "SRG":5,
                  "Libertarismus":6,
                  "Pro":7,
                  "Contra":8}
category_reverse_index = dict((y,x) for (x,y) in category_index.items())

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# Daten (Texte) in richtige Form bringen und Trennen für Tests
VALIDATION_SPLIT = 0.5

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
categories = to_categorical(np.asarray(labels))
print('Shape of data:', data.shape)
print('Shape of labels:', categories.shape)


indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
categories = categories[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = categories[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = categories[-nb_validation_samples:]
 
print('Anzahl jedes Frames in validation set')
print (y_train.sum(axis=0))
print (y_val.sum(axis=0))

Found 5320 unique tokens.
Shape of data tensor: (1034, 1000)
Shape of label tensor: (1034, 9)
Number of each frame in traing and validation set 
[167.  60.  40.  54.  18.  46.  54.  14.  64.]
[177.  48.  41.  48.  21.  43.  44.  26.  69.]


In [10]:
# Word2Vec-Layer vorbereiten
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
word_index = tokenizer.word_index
nb_words = min(MAX_NB_WORDS, len(word_index))+1

embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

embedding_layer = Embedding(embedding_matrix.shape[0], # or len(word_index) + 1
                            embedding_matrix.shape[1], # or EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Null word embeddings: 2836


## CNN-Modelle definieren

In [11]:
# CNN-Modell 1 definieren
model = Sequential()
model.add(embedding_layer)
model.add(Dropout(0.2))
model.add(Conv1D(300, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(150, 3, padding='valid',activation='relu',strides=2))
model.add(Conv1D(75, 3, padding='valid',activation='relu',strides=2))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(150,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(9,activation='sigmoid'))

model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 300)         1596300   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 300)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 499, 300)          270300    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 249, 150)          135150    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 124, 75)           33825     
_________________________________________________________________
flatten_1 (Flatten)          (None, 9300)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 9300)              0         
__________

In [12]:
# CNN-Modell 2 definieren
model_1 = Sequential()
model_1.add(embedding_layer)
model_1.add(Conv1D(250,3,padding='valid',activation='relu',strides=1))
model_1.add(GlobalMaxPooling1D())
model_1.add(Dense(250))
model_1.add(Dropout(0.2))
model_1.add(Activation('relu'))
model_1.add(Dense(9))
model_1.add(Activation('sigmoid'))
model_1.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1000, 300)         1596300   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 998, 250)          225250    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 250)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               62750     
_________________________________________________________________
dropout_4 (Dropout)          (None, 250)               0         
_________________________________________________________________
activation_1 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 2259      
__________

In [13]:
# CNN-Modell 3 definieren
embedding_layer = Embedding(embedding_matrix.shape[0], # or len(word_index) + 1
                            embedding_matrix.shape[1], # or EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)
# applying a more complex convolutional approach
convs = []
filter_sizes = [3,4,5]

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

for fsz in filter_sizes:
    l_conv = Conv1D(filters=128,kernel_size=fsz,activation='relu')(embedded_sequences)
    l_pool = MaxPooling1D(5)(l_conv)
    convs.append(l_pool)
    
l_merge = Concatenate(axis=1)(convs)
l_cov1= Conv1D(activation="relu", filters=128, kernel_size=5)(l_merge)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(activation="relu", filters=128, kernel_size=5)(l_pool1)
l_pool2 = MaxPooling1D(30)(l_cov2)
l_flat = Flatten()(l_pool2)
l_dense = Dense(128, activation='relu')(l_flat)
preds = Dense(9, activation='softmax')(l_dense)

model_2 = Model(sequence_input, preds)
model_2.compile(loss='categorical_crossentropy', 
                optimizer='rmsprop',
                metrics=['acc'])

model_2.summary()

model fitting - more complex convolutional neural network
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1000, 300)    1596300     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 998, 128)     115328      embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_6 (Conv1D)               (None, 997, 128)     153728      embedding_2[0][0]                
___________________________________________________

## CNN-Modelle ausführen

In [14]:
# Modell 1 ausführen und Scores anschauen
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=128)
score = model.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 517 samples, validate on 517 samples
Epoch 1/10
517/517 [==============================] - 8s 15ms/step - loss: 2.1679 - acc: 0.1760 - val_loss: 2.0532 - val_acc: 0.3424
Epoch 2/10
517/517 [==============================] - 8s 15ms/step - loss: 2.0334 - acc: 0.3230 - val_loss: 2.0170 - val_acc: 0.3424
Epoch 3/10
517/517 [==============================] - 8s 15ms/step - loss: 1.9824 - acc: 0.3230 - val_loss: 1.9921 - val_acc: 0.3424
Epoch 4/10
517/517 [==============================] - 8s 15ms/step - loss: 1.9858 - acc: 0.3191 - val_loss: 2.0106 - val_acc: 0.3424
Epoch 5/10
517/517 [==============================] - 8s 15ms/step - loss: 1.9397 - acc: 0.3230 - val_loss: 2.0110 - val_acc: 0.3424
Epoch 6/10
517/517 [==============================] - 8s 15ms/step - loss: 1.9349 - acc: 0.3230 - val_loss: 1.9546 - val_acc: 0.3424
Epoch 7/10
517/517 [==============================] - 8s 15ms/step - loss: 1.9011 - acc: 0.3230 - val_loss: 1.9454 - val_acc: 0.3424
Epoch 8/10
517/517 [===

In [15]:
# Modell 2 ausführen und Scores anschauen
model_1.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=128)
score = model_1.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 517 samples, validate on 517 samples
Epoch 1/10
517/517 [==============================] - 10s 19ms/step - loss: 2.0753 - acc: 0.2785 - val_loss: 2.0865 - val_acc: 0.3424
Epoch 2/10
517/517 [==============================] - 9s 18ms/step - loss: 1.9233 - acc: 0.3230 - val_loss: 2.0417 - val_acc: 0.3424
Epoch 3/10
517/517 [==============================] - 9s 18ms/step - loss: 1.8582 - acc: 0.3250 - val_loss: 2.0443 - val_acc: 0.3424
Epoch 4/10
517/517 [==============================] - 9s 18ms/step - loss: 1.8194 - acc: 0.3230 - val_loss: 1.9406 - val_acc: 0.3424
Epoch 5/10
517/517 [==============================] - 9s 18ms/step - loss: 1.7148 - acc: 0.3269 - val_loss: 2.0431 - val_acc: 0.3424
Epoch 6/10
517/517 [==============================] - 9s 18ms/step - loss: 1.7098 - acc: 0.3327 - val_loss: 1.9560 - val_acc: 0.3424
Epoch 7/10
517/517 [==============================] - 9s 18ms/step - loss: 1.6203 - acc: 0.3230 - val_loss: 1.9992 - val_acc: 0.3424
Epoch 8/10
517/517 [==

In [16]:
# Modell 3 ausführen und Scores anschauen
model_2.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10, batch_size=128)
score = model_2.evaluate(x_val, y_val, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 517 samples, validate on 517 samples
Epoch 1/10
517/517 [==============================] - 20s 38ms/step - loss: 2.0696 - acc: 0.2437 - val_loss: 2.0355 - val_acc: 0.2921
Epoch 2/10
517/517 [==============================] - 19s 37ms/step - loss: 1.9618 - acc: 0.3114 - val_loss: 1.9540 - val_acc: 0.2747
Epoch 3/10
517/517 [==============================] - 19s 37ms/step - loss: 1.8810 - acc: 0.3191 - val_loss: 1.9590 - val_acc: 0.3443
Epoch 4/10
517/517 [==============================] - 19s 37ms/step - loss: 1.8316 - acc: 0.3424 - val_loss: 2.0711 - val_acc: 0.3404
Epoch 5/10
517/517 [==============================] - 19s 37ms/step - loss: 1.7864 - acc: 0.3501 - val_loss: 2.0089 - val_acc: 0.3424
Epoch 6/10
517/517 [==============================] - 19s 37ms/step - loss: 1.7725 - acc: 0.3656 - val_loss: 2.0903 - val_acc: 0.2050
Epoch 7/10
517/517 [==============================] - 19s 37ms/step - loss: 1.7779 - acc: 0.3520 - val_loss: 1.8962 - val_acc: 0.3172
Epoch 8/10
517/5

### Einfache Tests mit bestem Modell (wird nicht mehr ausgeführt):